In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
import tqdm 

In [2]:
from sklearn.metrics import f1_score, precision_recall_curve


In [9]:
result_file = 'validate_MLP_results.txt'
fp = open(result_file)
lines = fp.readlines() 
lines = [x.strip() for x in lines]
fp.close()
print(lines[:10])

['features_val_non-covid_ct_scan_32.pkl,1,1,-3.046,2.548', 'features_val_non-covid_ct_scan_134.pkl,1,1,-2.223,1.886', 'features_val_non-covid_ct_scan_193.pkl,1,1,-2.026,1.733', 'features_val_non-covid_ct_scan_156.pkl,1,1,-0.093,0.139', 'features_val_covid_ct_scan_3.pkl,0,0,4.302,-4.387', 'features_val_covid_ct_scan_68.pkl,0,0,1.167,-1.100', 'features_val_non-covid_ct_scan_55.pkl,1,1,-2.775,2.383', 'features_val_non-covid_ct_scan_50.pkl,1,1,-1.633,1.332', 'features_val_non-covid_ct_scan_173.pkl,1,1,-1.232,1.069', 'features_val_non-covid_ct_scan_84.pkl,1,1,-2.386,2.085']


In [11]:
y_true = []
y_pred = [] 
y_prob = [] 

covid_ids = [] 
non_covid_ids = [] 

threshold = 0.5

for l in lines: 
    l_splits = l.split(',') 
    #print(l_splits)
    
    #change covid class label to 1 
    y_true.append(int(l_splits[1]))    
        
    prob0 = float(l_splits[3])
    prob1 = float(l_splits[4])

    score = np.exp(prob0)/(np.exp(prob0)+np.exp(prob1))
    
    pred2 = (score < threshold).astype(np.uint8)
    
    #y_pred.append(int(l_splits[2])) 
    y_pred.append(pred2) 
    y_prob.append(score)
    
    if pred2==0:
        covid_ids.append(l_splits[0])
    else: 
        non_covid_ids.append(l_splits[0])
    
y_pred = np.asarray(y_pred)
y_true = np.asarray(y_true)
y_prob = np.asarray(y_prob)

F1_1 = f1_score(y_true, y_pred, average='micro')
F1_2 = f1_score(y_true, y_pred, average='macro')
print(F1_1,F1_2)

0.9197860962566845 0.9179464677490127


In [17]:
fp = open('covid-MLP.csv','w')
len_covid = len(covid_ids) 
for i in range(len_covid):
    if i<len_covid-1:
        fp.write('{},'.format(covid_ids[i]))
    else:
        fp.write('{}'.format(covid_ids[i]))
fp.close() 


fp = open('non-covid-MLP.csv','w')
len_non_covid = len(non_covid_ids) 
for i in range(len_non_covid):
    if i<len_non_covid-1:
        fp.write('{},'.format(non_covid_ids[i]))
    else:
        fp.write('{}'.format(non_covid_ids[i]))
fp.close() 

